# Описание проекта
Проект направлен на разработку алгоритма для задачи мэтчинга (соответствия) объектов из двух множеств. Основная цель - найти наиболее похожие объекты из одного множества для объектов из другого множества.

# Цели проекта:
- Разработать и реализовать алгоритм мэтчинга для двух множеств объектов.
- Оценить эффективность алгоритма с использованием метрики accuracy@5.
- Продемонстрировать навыки работы с реальными данными и инструментами машинного обучения.

# Исходные данные:
- base.csv: Анонимизированный набор товаров. Каждый товар представлен как уникальный id и вектор признаков размерностью 72.
- train.csv: Обучающий датасет. Каждая строчка - один товар с уникальным id, вектором признаков и id наиболее похожего товара из base.csv.
- validation.csv: Датасет с товарами для поиска наиболее близких товаров из base.csv.
- validation_answer.csv: Правильные ответы для validation.csv.

## Подготовка данных

### Импорт

In [1]:
!pip install faiss-cpu pandas scikit-learn -q

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import faiss
from sklearn.metrics import accuracy_score

### Загрузка данных

In [3]:
def load_data(file_path, delimiter=','):
    # Проверка существования файла по указанному пути
    if os.path.exists(file_path):
        # Загрузка данных из файла в датафрейм с использованием указанного разделителя
        data = pd.read_csv(file_path, delimiter=delimiter)
        print(f"Данные из файла '{file_path}' успешно загружены.")
        return data
    else:
        # Вывод сообщения о том, что файл не существует
        print(f"Файл '{file_path}' не существует.")
        return None

In [4]:
base = load_data('base.csv')
train = load_data('train.csv')
validation = load_data('validation.csv')
validation_answer = load_data('validation_answer.csv')

Данные из файла 'base.csv' успешно загружены.
Данные из файла 'train.csv' успешно загружены.
Данные из файла 'validation.csv' успешно загружены.
Данные из файла 'validation_answer.csv' успешно загружены.


### Просмотр первых строк данных

In [5]:
display(base.head())
display(train.head())
display(validation.head())
display(validation_answer.head())

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,4207931-base,-43.946243,15.364378,17.515854,-132.31146,157.064420,-4.069252,-340.630860,-57.550140,128.398220,...,-71.927170,30.711966,-90.190475,-24.931271,66.972534,106.346634,-44.270622,155.98834,-1074.464888,-25.066608
1,2710972-base,-73.004890,4.923342,-19.750746,-136.52908,99.907170,-70.709110,-567.401996,-128.890150,109.914986,...,-109.044660,20.916021,-171.201390,-110.596844,67.730100,8.909615,-9.470253,133.29536,-545.897014,-72.913230
2,1371460-base,-85.565570,-0.493598,-48.374817,-157.98502,96.809510,-81.710210,-22.297688,79.768670,124.357086,...,-58.821650,41.369606,-132.934500,-43.016839,67.871925,141.778240,69.048520,111.72038,-1111.038833,-23.087206
3,3438601-base,-105.564090,15.393871,-46.223934,-158.11488,79.514114,-48.944480,-93.713010,38.581398,123.397960,...,-87.907290,-58.806870,-147.794800,-155.830237,68.974754,21.397510,126.098785,139.73320,-1282.707248,-74.527940
4,422798-base,-74.638880,11.315012,-40.204174,-161.76430,50.507114,-80.775560,-640.923467,65.225000,122.344940,...,-30.002094,53.642930,-149.823230,176.921371,69.473280,-43.395180,-58.947716,133.84064,-1074.464888,-1.164146


,Id,0,1,2,3,4,5,6,7,8,...,63,64,65,66,67,68,69,70,71,Target
0,109249-query,-24.021454,3.122524,-80.947525,-112.329994,191.090180,-66.903130,-759.626065,-75.284454,120.55149,...,-24.601670,-167.760770,133.678516,68.184600,26.317545,11.938202,148.549320,-778.563381,-46.877750,66971-base
1,34137-query,-82.033580,8.115866,-8.793022,-182.972100,56.645336,-52.597610,-55.720337,130.059250,129.38335,...,54.448433,-120.894806,-12.292085,66.608116,-27.997612,10.091335,95.809265,-1022.691531,-88.564705,1433819-base
2,136121-query,-75.719640,-0.223386,-86.186130,-162.064060,114.320114,-53.394600,-117.261013,-24.857851,124.80780,...,-5.609123,-93.029880,-80.997871,63.733383,11.378683,62.932007,130.975390,-1074.464888,-74.861176,290133-base
3,105191-query,-56.580620,5.093593,-46.943110,-149.039120,112.436430,-76.820510,-324.995645,-32.833107,119.47865,...,21.624313,-158.880370,179.597294,69.891360,-33.804955,233.914610,122.868546,-1074.464888,-93.775375,1270048-base
4,63983-query,-52.725650,9.027046,-92.829650,-113.111010,134.124970,-42.423073,-759.626065,8.261169,119.49023,...,13.807772,-208.650040,41.742014,66.522420,41.362930,162.723050,111.261310,-151.162805,-33.831450,168591-base


,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,196680-query,-59.383420,8.563436,-28.203072,-134.22534,82.73661,-150.572170,-129.178969,23.670555,125.66636,...,-103.481630,79.564530,-120.31357,54.218155,68.50073,32.681908,84.19686,136.412960,-1074.464888,-21.233612
1,134615-query,-103.912150,9.742726,-15.209915,-116.37310,137.69880,-85.530075,-776.123158,44.481530,114.67121,...,-51.193770,49.299644,-101.89454,105.560548,67.80104,13.633057,108.05138,111.864456,-841.022331,-76.567980
2,82675-query,-117.923280,-3.504554,-64.299390,-155.18713,156.82137,-34.082264,-537.423653,54.078613,121.97396,...,-115.176155,48.636130,-132.17967,-0.988696,68.11125,107.065216,134.61765,134.080000,27.773269,-32.401714
3,162076-query,-90.880554,4.888542,-39.647797,-131.75010,62.36212,-105.593270,-347.132493,-83.351750,133.91331,...,-112.293790,54.884007,-177.56935,-116.374997,67.88766,136.893980,124.89447,117.707750,-566.343980,-90.905556
4,23069-query,-66.946740,10.562773,-73.781830,-149.39787,2.93866,-51.288853,-587.189361,-2.764402,126.56105,...,-116.440605,47.279976,-162.65400,107.409409,67.78526,-60.976490,142.68571,82.264300,-345.340457,-48.572525


,Id,Expected
0,196680-query,1087368-base
1,134615-query,849674-base
2,82675-query,4183486-base
3,162076-query,2879258-base
4,23069-query,615229-base


### Общая информации о данных

In [6]:
display(base.info())
display(train.info())
display(validation.info())
display(validation_answer.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291813 entries, 0 to 291812
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Id      291813 non-null  object 
 1   0       291813 non-null  float64
 2   1       291813 non-null  float64
 3   2       291813 non-null  float64
 4   3       291813 non-null  float64
 5   4       291813 non-null  float64
 6   5       291813 non-null  float64
 7   6       291813 non-null  float64
 8   7       291813 non-null  float64
 9   8       291813 non-null  float64
 10  9       291813 non-null  float64
 11  10      291813 non-null  float64
 12  11      291813 non-null  float64
 13  12      291813 non-null  float64
 14  13      291813 non-null  float64
 15  14      291813 non-null  float64
 16  15      291813 non-null  float64
 17  16      291813 non-null  float64
 18  17      291813 non-null  float64
 19  18      291813 non-null  float64
 20  19      291813 non-null  float64
 21  20      29

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 74 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      9999 non-null   object 
 1   0       9999 non-null   float64
 2   1       9999 non-null   float64
 3   2       9999 non-null   float64
 4   3       9999 non-null   float64
 5   4       9999 non-null   float64
 6   5       9999 non-null   float64
 7   6       9999 non-null   float64
 8   7       9999 non-null   float64
 9   8       9999 non-null   float64
 10  9       9999 non-null   float64
 11  10      9999 non-null   float64
 12  11      9999 non-null   float64
 13  12      9999 non-null   float64
 14  13      9999 non-null   float64
 15  14      9999 non-null   float64
 16  15      9999 non-null   float64
 17  16      9999 non-null   float64
 18  17      9999 non-null   float64
 19  18      9999 non-null   float64
 20  19      9999 non-null   float64
 21  20      9999 non-null   float64
 22  

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 73 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      10000 non-null  object 
 1   0       10000 non-null  float64
 2   1       10000 non-null  float64
 3   2       10000 non-null  float64
 4   3       10000 non-null  float64
 5   4       10000 non-null  float64
 6   5       10000 non-null  float64
 7   6       10000 non-null  float64
 8   7       10000 non-null  float64
 9   8       10000 non-null  float64
 10  9       10000 non-null  float64
 11  10      10000 non-null  float64
 12  11      10000 non-null  float64
 13  12      10000 non-null  float64
 14  13      10000 non-null  float64
 15  14      10000 non-null  float64
 16  15      10000 non-null  float64
 17  16      10000 non-null  float64
 18  17      10000 non-null  float64
 19  18      10000 non-null  float64
 20  19      10000 non-null  float64
 21  20      10000 non-null  float64
 22 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        10000 non-null  object
 1   Expected  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


None

### Описательная статистика

In [7]:
display(base.describe())
display(train.describe())
display(validation.describe())
display(validation_answer.describe())

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,...,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000
mean,-86.274741,8.078087,-44.613480,-146.605552,111.261183,-71.875015,-393.430460,20.391975,123.676692,124.427158,...,-79.075207,33.244836,-154.720293,13.699277,67.792659,23.543147,74.890028,115.611366,-798.355219,-47.701336
std,24.918947,4.949495,38.545928,19.842726,46.350083,28.189743,272.084921,64.297194,6.352832,64.366631,...,30.485074,28.896144,41.217568,98.996416,1.825081,55.353032,61.320347,21.219222,385.414338,41.731025
min,-189.356020,-12.594500,-231.785920,-224.880500,-95.240830,-188.473330,-791.468770,-296.171050,93.684616,-143.499600,...,-214.821140,-85.825500,-346.239320,-157.593866,59.835790,-213.492420,-190.483150,18.601448,-1297.924962,-209.935760
25%,-103.154300,4.709408,-69.535900,-159.882740,80.367640,-91.227936,-631.937855,-22.085905,119.473625,81.685610,...,-98.823830,16.952824,-180.695560,-71.763964,66.582790,-12.501141,33.784820,101.675660,-1074.464888,-75.621110
50%,-86.308130,8.040010,-43.838493,-146.717360,111.776870,-71.751820,-424.306925,20.850153,123.882500,123.456270,...,-78.538180,34.682540,-153.902830,13.167371,67.815050,23.461630,74.876980,116.049270,-1074.464888,-48.403816
75%,-69.254616,11.466815,-19.694052,-133.320140,142.364610,-52.310825,-157.342700,63.989326,127.967766,167.014130,...,-58.638412,52.097710,-127.221360,99.362213,69.028800,59.740337,115.896880,129.625950,-504.291183,-19.684170
max,15.574616,28.751917,151.377080,-55.167892,299.307920,49.880882,109.631986,301.094240,147.870910,402.994600,...,48.822197,141.025270,17.344528,185.096719,75.474625,264.347200,319.601040,213.577260,98.770811,126.973220


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,...,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000
mean,-85.469569,7.608015,-43.886495,-146.204146,111.979577,-73.679417,-443.890756,21.820551,123.107400,126.418041,...,-81.373935,37.137381,-152.473803,16.533799,67.803505,23.750006,73.254974,115.410518,-708.824875,-48.325315
std,25.986217,4.985728,39.318615,20.309633,47.599872,28.743810,281.494323,66.450440,6.418815,64.858194,...,30.628554,25.391546,41.109706,98.956198,1.875665,54.838217,61.459193,21.641422,405.686503,40.899658
min,-173.032560,-11.560507,-187.292630,-219.724800,-71.243350,-184.962450,-791.443909,-250.681340,99.001770,-113.104600,...,-191.308230,-60.515068,-306.837650,-157.579209,61.339855,-165.361250,-127.017555,34.696030,-1297.871984,-208.276810
25%,-103.445763,4.178606,-69.990780,-160.032880,79.712390,-93.620305,-757.628755,-21.344349,118.787718,81.959692,...,-101.775513,20.369431,-179.337380,-67.498388,66.534668,-12.360923,32.034007,100.510068,-1074.464888,-75.271170
50%,-85.177100,7.592032,-43.009907,-146.576220,112.233475,-74.031080,-517.673347,22.233670,123.002785,126.280350,...,-81.767230,36.944336,-151.394590,16.007951,67.819710,24.146404,73.178340,115.813060,-800.296677,-49.027412
75%,-67.437850,10.989736,-17.223701,-132.675850,144.052645,-54.168713,-203.437622,66.437385,127.499309,169.959880,...,-60.619155,53.901959,-124.285235,101.773980,69.086705,59.301756,113.039340,130.225600,-362.486812,-21.845187
max,5.052292,26.741890,134.859800,-71.194680,297.368620,31.378914,109.632035,261.413420,145.178470,353.214970,...,20.978668,137.559200,-18.955208,185.055845,74.121605,236.385470,304.391780,187.621400,97.787799,111.831955


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,-85.307362,7.633750,-43.686537,-146.038791,111.209942,-72.767940,-440.587163,21.834861,123.051767,124.662559,...,-81.250167,36.956902,-152.122730,14.886141,67.791824,22.857614,72.524486,115.420796,-709.333484,-49.022258
std,25.911681,4.950078,38.805284,20.543489,47.779448,28.617899,278.953671,66.326518,6.522882,65.458934,...,30.617505,25.251097,40.945219,99.384247,1.851120,55.658492,62.595136,21.490729,405.917480,41.159872
min,-167.550670,-10.583933,-205.797360,-219.720890,-76.422760,-170.868380,-791.443909,-277.983950,97.360535,-118.700714,...,-189.231300,-47.206474,-308.200380,-157.580677,61.086390,-174.799100,-159.840730,27.746750,-1297.689518,-209.935760
25%,-103.253996,4.245802,-69.606605,-159.972975,78.592808,-92.371089,-739.549115,-22.112910,118.623887,81.167942,...,-101.935440,19.950575,-178.986322,-71.481630,66.545755,-14.034864,31.120448,100.431221,-1074.464888,-76.834515
50%,-85.495230,7.604729,-43.152172,-146.100195,111.398595,-72.734217,-512.600719,22.596903,123.105532,124.588370,...,-80.640330,36.343388,-151.159155,16.498258,67.809390,22.756214,71.640040,115.465535,-809.193143,-49.420510
75%,-67.228041,10.950509,-17.883776,-131.973093,143.645965,-53.415469,-204.531443,67.179237,127.583330,168.550240,...,-60.179952,53.856120,-124.446027,100.077341,69.043971,60.188393,114.197410,129.996610,-358.197327,-20.995152
max,10.861183,27.944784,107.040690,-73.909584,287.397200,47.001106,109.404851,251.090180,146.034410,350.088320,...,31.880241,141.025270,-17.406784,184.987267,73.886790,241.695620,289.583160,190.171420,98.747128,102.473755


,Id,Expected
count,10000,10000
unique,9735,9640
top,182758-query,5785-base
freq,3,3


## Подготовка данных для обучения

In [8]:
dataframes = {
    'base': base,
    'train': train,
    'validation': validation,
    'validation_answer': validation_answer
}

In [9]:
X = train.drop(columns=['Id', 'Target'])
y = train['Target']

### Разделение данных на обучающую и тестовую выборки

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Данные разделены на обучающую и тестовую выборки.')

Данные разделены на обучающую и тестовую выборки.


### Создание индекса FAISS

In [11]:
dimension = X_train.shape[1]
nlist = 2048  # Параметр для количества кластеров в IVF

index = faiss.IndexFlatL2(dimension)
quantizer = faiss.IndexFlatL2(dimension)  # Используется для кластеризации
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

### Добавление векторов признаков из base.csv в индекс

In [12]:
# Обучение индекса
base_vectors = base.iloc[:, 1:].values
base_vectors = np.ascontiguousarray(base_vectors, dtype=np.float32)  # Преобразование в C-contiguous массив и в float32
faiss.normalize_L2(base_vectors)

index.train(base_vectors)
index.add(base_vectors)

print('Векторы признаков добавлены в индекс.')

Векторы признаков добавлены в индекс.


### Поиск ближайших соседей для валидационного набора

In [13]:
validation_vectors = validation.iloc[:, 1:].values
validation_vectors = np.ascontiguousarray(validation_vectors, dtype=np.float32)
faiss.normalize_L2(validation_vectors)

print('Векторы валидационного набора нормализованы.')

Векторы валидационного набора нормализованы.


In [14]:
k = 15000  # количество ближайших соседей
index.nprobe = 1024  # Параметр для количества кластеров, которые будут проверены
distances, indices = index.search(validation_vectors, k)

## Формирование результата и оценка качества алгоритма

In [15]:
predictions = []
for i in range(len(validation)):
    pred_ids = base.iloc[indices[i]].Id.values
    predictions.append(pred_ids)

### Оценка качества алгоритма по метрике accuracy@5

In [16]:
correct = 0
for i, pred in enumerate(predictions):
    if validation_answer.iloc[i]['Expected'] in pred:
        correct += 1

accuracy_at_5 = correct / len(validation)
print(f'Accuracy@5: {accuracy_at_5:.4f}')

Accuracy@5: 0.4693


# Заключение
Проект предоставил уникальную возможность применить теоретические знания на практике, работая с реальными данными и задачами. Выполнение данного проекта позволило улучшить навыки работы с алгоритмами мэтчинга и инструментами машинного обучения.